In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from qiskit_metal import MetalGUI, designs
from qiskit_metal.qlibrary.qubits.transmon_pocket_6 import TransmonPocket6

In [16]:
%metal_heading Create a Transmon Qubit in Qiskit Metal

In [ ]:
design = designs.DesignPlanar({}, True)
gui = MetalGUI(design)

In [ ]:
conn_pads = dict(
    connection_pads = dict(
        readout = dict(loc_W=0, loc_H=-1),
        coupler1 = dict(loc_W=-1, loc_H=1),
        coupler2 = dict(loc_W=1, loc_H=1)
    )
)
q1 = TransmonPocket6(design, "Q1", options=dict(**conn_pads))

gui.rebuild()
gui.autoscale()

In [17]:
%metal_heading Render your design in QElmerRenderer

In [ ]:
from qiskit_metal.renderers.renderer_elmer.elmer_renderer import QElmerRenderer

# Instantiate the Elmer renderer
elmer_renderer = QElmerRenderer(design)

# Elmer renderer uses the Gmsh renderer to
# generate a mesh for the design
# Set initial parameters for meshing in Gmsh (IMPORTANT step!!)
elmer_renderer.gmsh.options.mesh.min_size = "3um"
elmer_renderer.gmsh.options.mesh.max_size = "30um"

In [ ]:
# Render the design
elmer_renderer.render_design(open_pins=[("Q1", "coupler1"), ("Q1", "coupler2"), ("Q1", "readout")],
                             skip_junctions=True)

In [ ]:
# View the generated mesh
elmer_renderer.launch_gmsh_gui()

In [ ]:
# Export the generated mesh
elmer_renderer.export_mesh()

In [ ]:
# Add a solution setup to solve for the capacitance matrix
elmer_renderer.add_solution_setup('capacitance')

In [ ]:
# Run the simulation in ElmerFEM
elmer_renderer.run('capacitance')

In [ ]:
# Display the capacitnce matrix obtained after the simulation
elmer_renderer.capacitance_matrix

In [ ]:
# Run this, and in the Gmsh window,
# Right click on an empty area, click on "Toggle mesh visibility"
# Next, go to: Tools -> Visibility, select "Physical groups" in drop down menu
# Select "Volume 2", click on apply
# On the left pane, click on "Post-processing", and select the views that you want to observe
elmer_renderer.display_post_processing_data()

In [ ]:
elmer_renderer.save_capacitance_matrix("cap_matrix.txt")
elmer_renderer.close()

In [18]:
%metal_heading Perform LOM 2.0 Analysis

In [ ]:
import numpy as np

from qiskit_metal.renderers.renderer_elmer.elmer_renderer import load_capacitance_matrix_from_file
from qiskit_metal.analyses.quantization.lom_core_analysis import CompositeSystem, Cell, Subsystem 

from scipy.constants import speed_of_light as c_light

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
cap_matrix = load_capacitance_matrix_from_file("cap_matrix.txt")

In [ ]:
# Define cells
opt1 = dict(
    node_rename = {'Q1_coupler1_connector_pad': 'coupler1', 'Q1_coupler2_connector_pad': 'coupler2', 'Q1_readout_connector_pad': 'readout'}, 
    cap_mat = cap_matrix,
    ind_dict = {('Q1_pad_top', 'Q1_pad_bot'): 12},  # junction inductance in nH
    jj_dict = {('Q1_pad_top', 'Q1_pad_bot'): 'j1'},
    cj_dict = {('Q1_pad_top', 'Q1_pad_bot'): 2}, # junction capacitance in fF
)
cell_1 = Cell(opt1)

In [ ]:
# Define subsystems
# subsystem 1: Transmon
transmon = Subsystem(name='transmon1_sys', sys_type='TRANSMON', nodes=['j1'])

# subsystem 2: Coupler 1
q_opts = dict(
    f_res = 7.4, # resonator dressed frequency in GHz
    Z0 = 50, # characteristic impedance in Ohm
    vp = 0.404314 * c_light # phase velocity 
)
coup1 = Subsystem(name='coup1_sys', sys_type='TL_RESONATOR', nodes=['coupler1'], q_opts=q_opts)

# subsystem 3: Coupler 2
q_opts = dict(
    f_res = 7.2, # resonator dressed frequency in GHz
    Z0 = 50, # characteristic impedance in Ohm
    vp = 0.404314 * c_light # phase velocity 
)
coup2 = Subsystem(name='coup2_sys', sys_type='TL_RESONATOR', nodes=['coupler2'], q_opts=q_opts)

# subsystem 4: Readout
q_opts = dict(
    f_res = 7, # resonator dressed frequency in GHz
    Z0 = 50, # characteristic impedance in Ohm
    vp = 0.404314 * c_light # phase velocity 
)
readout = Subsystem(name='readout_sys', sys_type='TL_RESONATOR', nodes=['readout'], q_opts=q_opts)

In [ ]:
composite_sys = CompositeSystem(
    subsystems=[transmon, coup1, coup2, readout], 
    cells=[cell_1], 
    grd_node='ground_plane',
    nodes_force_keep=['coupler1', 'coupler2', 'readout']
)

In [ ]:
cg = composite_sys.circuitGraph()
print(cg)

In [ ]:
hilbertspace = composite_sys.create_hilbertspace()
print(hilbertspace)

In [ ]:
hilbertspace = composite_sys.add_interaction()
hilbertspace.hamiltonian()

In [ ]:
hamiltonian_results = composite_sys.hamiltonian_results(hilbertspace, evals_count=30)

In [ ]:
composite_sys.compute_gs()

In [ ]:
transmon.h_params

In [ ]:
gui.main_window.close()